# LLM API — Getting Started

This notebook shows how to call the **GPT-OSS** model hosted on `hub.qazcode.ai`.

The server uses the **OpenAI-compatible API**, so you can use the `openai` Python library or plain `requests`.

## 1. Install dependencies

In [44]:
#!pip install openai

## 2. Configuration

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
API_KEY = os.environ.get('API_KEY')
HUB_URL = os.environ.get('HUB_URL')

if not API_KEY:
    raise ValueError("API_KEY is not set in environment variables")

In [21]:
from openai import OpenAI

client = OpenAI(
    base_url=HUB_URL,
    api_key=API_KEY,  # replace with your key
)

MODEL = "oss-120b"

## 3. Basic Chat Completion

In [22]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hello! Who are you?"}
    ],
)

print(response.choices[0].message.content)

Hello! I’m ChatGPT — an AI language model created by OpenAI. I’m here to help answer questions, brainstorm ideas, explain concepts, or just chat about whatever’s on your mind. How can I assist you today?


## 4. System Prompt + User Message

In [23]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a medical diagnosis assistant. Given patient symptoms, suggest the most probable diagnosis with an ICD-10 code."
        },
        {
            "role": "user",
            "content": "Patient presents with fever, dry cough, and shortness of breath lasting 5 days."
        }
    ],
)

print(response.choices[0].message.content)

**Probable Diagnosis:**  
- **COVID‑19, virus identified**  

**ICD‑10‑CM Code:** **U07.1**  

**Rationale:**  
- The triad of fever, dry cough, and dyspnea over several days is classic for an acute viral lower‑respiratory infection.  
- Since the onset is recent (5 days) and the cough is non‑productive, COVID‑19 is the leading consideration, especially in the context of ongoing community transmission.  
- Influenza and acute viral bronchitis present similarly, but they are coded separately (J10.1‑J10.9 for influenza, J20.9 for acute bronchitis). If a rapid antigen or PCR test for SARS‑CoV‑2 returns positive, U07.1 is the appropriate code. If testing is negative, the differential would shift toward influenza (J10.1‑J10.9) or unspecified viral pneumonia (J12.9).  

**Suggested next steps:**  

1. **Obtain a nasopharyngeal swab for SARS‑CoV‑2 PCR or rapid antigen test** (and consider influenza testing if COVID‑19 is negative).  
2. **Assess oxygen saturation** and consider bedside pulse 

## 5. Structured JSON Output

In [24]:
import json

SYSTEM_PROMPT = """You are a clinical decision support system.
Given patient symptoms, return a JSON object:
{
  "diagnoses": [
    {"rank": 1, "icd_code": "...", "name": "...", "explanation": "..."},
    ...
  ]
}
Return top 3 diagnoses ranked by likelihood. Respond with JSON only."""

symptoms = "Severe headache, nausea, vomiting, neck stiffness, sensitivity to light."

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": symptoms},
    ],
)

result = json.loads(response.choices[0].message.content)
for d in result["diagnoses"]:
    print(f"[{d['rank']}] {d['icd_code']} — {d['name']}")
    print(f"    {d['explanation']}\n")

[1] G00.9 — Acute bacterial meningitis, unspecified
    Classic triad of severe headache, neck stiffness, and photophobia, often accompanied by nausea/vomiting and fever. Rapid progression and high morbidity make bacterial meningitis the most likely diagnosis.

[2] I60.9 — Subarachnoid hemorrhage, unspecified
    Sudden onset severe headache (“thunderclap”), neck stiffness, and photophobia can also arise from blood in the subarachnoid space. Vomiting is common. Imaging is required to differentiate from meningitis.

[3] A87.9 — Viral meningitis, unspecified
    Viral meningitis presents similarly with headache, neck stiffness, photophobia, and gastrointestinal symptoms, but typically has a milder course than bacterial meningitis. It remains a common differential.



In [25]:
import requests

response = requests.post(
    f"{HUB_URL}/chat/completions",
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    },
    json={
        "model": "oss-120b",
        "messages": [{"role": "user", "content": "Hello!"}],
    },
)

print(response.json()["choices"][0]["message"]["content"])

Hello! 👋 How can I assist you today?


In [24]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1")

In [23]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(
#     model="oss-120b",
#     api_key=API_KEY,
#     base_url=HUB_URL,
#     temperature=0
# )

# print(llm.invoke("Hello!").content)

In [10]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="ai-forever/ru-en-RoSBERTa",
    openai_api_base="http://localhost:8081/v1",
    openai_api_key="unused",
    check_embedding_ctx_length=False,
)

In [11]:
from pathlib import Path
from langchain_chroma import Chroma

# Use an absolute path so the DB is writable (avoids "readonly database" error).
# If it still fails, use e.g. persist_dir = Path("/tmp/chroma_langchain_db").
persist_dir = (Path.cwd() / "chroma_langchain_db").resolve()
persist_dir.mkdir(parents=True, exist_ok=True)

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory=str(persist_dir),
)

In [12]:
import json
from pathlib import Path
from langchain_core.documents import Document

data_dir = Path("../data/test_set")

docs = []
for json_file in sorted(data_dir.glob("*.json")):
    with open(json_file) as f:
        data = json.load(f)
    docs.append(Document(
        page_content=data["text"],
        metadata={
            "protocol_id": data["protocol_id"],
            "icd_codes": data["icd_codes"],
            "source_file": data["source_file"],
        },
    ))

print(f"Loaded {len(docs)} documents")
docs[0]
print(f"Total characters: {len(docs[0].page_content)}")

Loaded 221 documents
Total characters: 12844


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 15250 sub-documents.


In [14]:
BATCH_SIZE = 1000
document_ids = []
for i in range(0, len(all_splits), BATCH_SIZE):
    batch = all_splits[i : i + BATCH_SIZE]
    document_ids.extend(vector_store.add_documents(documents=batch))
    print(f"Added batch {i // BATCH_SIZE + 1} ({len(batch)} docs)")

print(f"\nTotal documents added: {len(document_ids)}")

Added batch 1 (1000 docs)
Added batch 2 (1000 docs)
Added batch 3 (1000 docs)
Added batch 4 (1000 docs)
Added batch 5 (1000 docs)


KeyboardInterrupt: 

In [15]:
print(vector_store._collection.count())

5000


In [16]:
# Top 20 candidates by similarity score (same idea as docs; feed these to the agent via retrieve_context)
query = "Когда идём, его качает, будто пол уплывает, пару раз чуть не упал."
results = vector_store.similarity_search_with_score(query, k=20)
for res, score in results:
    print(f"* [SIM={score:.3f}] {res.page_content[:300]}... [{res.metadata}]")

* [SIM=1.144] миелограмма);  Определение гаммаглютамилтранспептидазы;  УЗИ мошонки, УЗИ придаточных пазух носа, УЗИ щитовидной железы, УЗИ предстательной железы и яичек – по показаниям;  ЭЭГ (рутинное), ЭЭГ (видеомониторирование трехчасовое) – при подозрении на поражение со стороны цнс, при судорогах;  Опреде... [{'protocol_id': 'p_3bdc46982f', 'source_file': 'Остеогенная Саркома (Euramos 1Coss).pdf', 'start_index': 24782, 'icd_codes': ['C30', 'C40.0', 'C40.1', 'C40.2', 'C40.3', 'C41.2', 'C41.3', 'C41.4']}]
* [SIM=1.166] «холод», Повторяет сложные слова «мама», «папа», одежду, обувь «голод», «усталость» «революция», обращенные к родителям «конференция» Понимает значение слов По просьбе показывает Называет 3 числа Отвечает на вопросы о «нельзя», «дай» одну часть тела значении мебели, книг, посуды Социальные навыки Ул... [{'icd_codes': ['A84', 'A87', 'A89', 'B67.9', 'B69.0', 'B94', 'D16.6', 'D32.0', 'D32.1', 'D33.0', 'D33.1', 'D33.3', 'D33.4', 'D36.1', 'F48', 'F80', 'F84', 'F90', 'F9

In [17]:
from pydantic import BaseModel, Field
from typing import Annotated

class AgentResponse(BaseModel):
    ICD_10_code: Annotated[
        list[str],
        Field(
            description="ICD-10 codes that identify diseases or conditions.",
            min_length=3,
            max_length=3
        )
    ]

In [25]:
from langchain.tools import tool

@tool
def get_top3_protocols(query: str) -> str:
    """Retrieve the top 3 most relevant clinical protocol excerpts for the given patient symptoms query. Call this first with the user's symptoms."""
    results = vector_store.similarity_search_with_score(query, k=3)
    parts = []
    for i, (doc, score) in enumerate(results, 1):
        parts.append(f"[Match {i}, score={score:.3f}] Source: {doc.metadata}\nContent: {doc.page_content}")
    return "\n\n---\n\n".join(parts)

from langchain.agents import create_agent

tools = [get_top3_protocols]
prompt = (
    "You are a medical diagnosis assistant. "
    "You MUST call get_top3_protocols first with the patient's symptoms (the user message) to retrieve the top 3 relevant clinical protocol excerpts. "
    "Based only on that retrieved context, respond with exactly 3 ICD-10 codes (most probable diagnoses, ranked by likelihood). "
    "Your final response must be a valid JSON object with this exact structure: {\"ICD_10_code\": [\"code1\", \"code2\", \"code3\"]}. "
    "Example: {\"ICD_10_code\": [\"R42\", \"G43.1\", \"F41.0\"]}. No other text, only the JSON."
)
agent = create_agent(llm, tools, system_prompt=prompt)

In [26]:
import json
import re
query = "Когда идём, его качает, будто пол уплывает, пару раз чуть не упал."
result = agent.invoke({"messages": [{"role": "user", "content": query}]})
last_message = result["messages"][-1]
content = last_message.content if hasattr(last_message, "content") else str(last_message)
# Extract JSON (strip markdown code block if present)
raw = re.sub(r"^```(?:json)?\s*", "", content.strip()).rstrip("`")
try:
    parsed = AgentResponse.model_validate_json(raw)
except Exception:
    start = raw.find("{")
    if start != -1:
        depth, end = 0, start
        for i, c in enumerate(raw[start:], start):
            if c == "{": depth += 1
            elif c == "}": depth -= 1
            if depth == 0: end = i; break
        parsed = AgentResponse.model_validate_json(raw[start:end+1])
    else:
        raise
print("Structured output:", parsed)
print("Top 3 ICD-10 codes:", parsed.ICD_10_code)

Structured output: ICD_10_code=['G11', 'G80', 'G81']
Top 3 ICD-10 codes: ['G11', 'G80', 'G81']
